In [1]:
!pip3 install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=e520d831e35b11d2e48a19b981485fb097f81578cc2a0f11e5167c5cd2109207
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


In [2]:
import requests
from bs4 import BeautifulSoup 
import string
import sys
sys.setrecursionlimit(1500) # Increase recursion limit
from collections import Counter
import nltk 
import re
import matplotlib.pyplot as plt
from urllib.request import urlopen
import requests
import pandas as pd
from urllib.parse import urlparse

In [3]:
import pandas as pd
import wikipedia
wikipedia.set_lang("ta")

In [37]:
people_df=pd.read_csv()#path to the CSV file

In [27]:
names=[] #Names of People

In [29]:
def get_name(string):
    
    source = requests.get(f"https://ta.wikipedia.org/wiki/" + string) # Get access to the article
    soup = BeautifulSoup(source.text, 'html.parser')
    name = soup.find('h1').text
    return name

In [30]:
def get_values(string):
    birthdate = None
    source = requests.get(f"https://ta.wikipedia.org/wiki/" + string) # Get access to the article
    soup = BeautifulSoup(source.text, 'html.parser')
    if(soup.find('table',attrs={'class':'infobox vcard'})==None):
      table_detail= soup.find('table',attrs={'class':'infobox'})
    else:
      table_detail= soup.find('table',attrs={'class':'infobox vcard'})
    table_row_list = table_detail.findAll('tr')
    for tr in table_row_list:
        try:
            if ((tr.th.text).strip() == 'பிறப்பு'):
                if tr.td.findAll('span', {'class': 'bday'}):
                    birthdate = tr.td.find('span', {'class': 'bday'}).text
        except AttributeError or TypeError:
            continue
    df=pd.read_html(str(table_detail))
    list1=df[0].values.tolist()
    del list1[0]
    dict_values = {}
    for elem in list1:
      if elem[0] in dict_values:
        dict_values[elem[0]].append(elem[1])
      else:
        dict_values[elem[0]] = [elem[1]]
    dict_values['birthdate']=birthdate
    return dict_values

In [31]:
def get_content(string):
  wiki = wikipedia.page(string)
  text = wiki.content
  text = re.sub(r'==.*?==+', '', text)
  text = text.replace('\n', '.')
  text = re.sub(r'\.+', ".", text)
  text = re.sub(r"[\([{'?\})\]]", "", text)
  #text = re.sub("[\a-zA-Z]+", " ",text)

  return text

In [ ]:
get_content('மில்ட்டன் பிரீட்மன்')

In [33]:
def get_summary(string):
  summary=wikipedia.summary(string)
  summary = re.sub(r"^$.\n" ,"\n",summary)
  summary = summary.replace("\'s","'s")
  summary = summary.replace('\xa0','')
  summary = re.sub(r'\n+', "\n", summary)
  summary = re.sub(r"[\([{'?\})\]]", "", summary)
  #text = re.sub("[\a-zA-Z]+", " ",summary)

  return summary

In [ ]:
get_summary('மில்ட்டன் பிரீட்மன்')

In [38]:
for name in names1:
  if name not in people_df['name'].unique():
    name=get_name(name)
    dict_values= get_values(name)

    if('பிறப்பு' in dict_values):
        born_year = dict_values['birthdate']

    if('துறை' in dict_values):  
      area_work=','.join(dict_values['துறை'][0].split(' '))
      area_work = re.sub(r'\,+', ",", area_work)
    else:
      area_work=" "

    if('அறியப்பட்டது' in dict_values):
      known_for = ','.join(dict_values['அறியப்பட்டது'][0].split(' '))
      known_for = re.sub(r'\,+', ",", known_for)
    elif ('அறியப்படுவது'in dict_values):
      known_for = ','.join(dict_values['அறியப்படுவது'][0].split(' '))
      known_for = re.sub(r'\,+', ",", known_for)
    elif ('பங்களிப்புகள்'in dict_values):
      known_for = ','.join(dict_values['பங்களிப்புகள்'][0].split(' '))
      known_for = re.sub(r'\,+', ",", known_for)  
    else:
      known_for=' '

    if('பரிசுகள்' in dict_values):
      awards=','.join(dict_values['பரிசுகள்'][0].split(')'))
      awards=awards.replace('(', "")
    elif('விருதுகள்' in dict_values):
      awards=','.join(dict_values['விருதுகள்'][0].split(')'))
      awards=awards.replace('(', "")
    else:
      awards= " "
    count=55
    content= get_content(name)
    summary= get_summary(name)
    url_page ='https://ta.wikipedia.org/wiki/'+ '_'.join(name.split(' '))
    series_obj=pd.Series([count,name,born_year,area_work,known_for, awards, summary, content,url_page],index=people_df.columns)
    count+=1
    people_df = people_df.append(  series_obj,
                        ignore_index=True)




In [39]:
people_df.to_csv('famous_people.csv')

In [41]:
!cp famous_people.csv --path_to_save